# Latent space visualization

In [ ]:
import logging
from pathlib import Path
from pprint import pprint
from src.nb_imports import *


from fastai.losses import MSELossFlat
from fastai.learner import Learner


import fastai
# from fastai.tabular.all import *

from fastai.basics import *
from fastai.callback.all import *
from fastai.torch_basics import *
from fastai.data.all import *

# import fastai.callback.hook # Learner.summary

import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import vaep.io_images
from vaep.models import ae
from vaep.transform import VaepPipeline
from vaep.io import datasplits
from vaep.io.dataloaders import get_dls

import src
import src.analyzers as analyzers
from src import config
from src.logging import setup_logger
logger = setup_logger(logger=logging.getLogger('vaep'))
logger.info("Experiment 03 - Analysis of latent spaces and performance comparisions")

figures = {}  # collection of ax or figures

Papermill script parameters

In [ ]:
n_peptides = 50
data = 'data/msinstrument_in_QE4'
epochs_max = 30

Some argument transformations

In [ ]:
args = config.Config()
args.data = Path(data)
args.epochs_max = epochs_max

## Load data

In [ ]:
data = datasplits.DataSplits.from_folder(args.data)

data is loaded in long format

In [ ]:
data.train_X.sample(5)

## Initialize Comparison

- replicates idea for truely missing values: Define truth as by using n=3 replicates to impute
  each sample
- real test data: Not used for predictions or early stopping.

In [ ]:
# data.test_y
test_pred_real_na = data.interpolate('test_X') # "gold standard"
# data.test_y.index.difference(test_predictions_real_na.index) # empty
# test_predictions_real_na.compare(data.test_y) # some wiered bug, maybe due to multi-index?
# are indices exactly the same? seems so
# sorting values and visual inspection: the values seem to be the same
# conclusions: floating point differences, which can be savely ignored
assert abs((test_pred_real_na - data.test_y).sum()) < 0.000001


test_pred_real_na = test_pred_real_na.to_frame(name='replicated')
# test_pred_real_na = data.test_y
test_pred_real_na.sort_index(inplace=True)
test_pred_real_na

In [ ]:
test_pred_observed = data.test_X.to_frame('measured')
test_pred_observed.sort_index(inplace=True)

In [ ]:
ana_train_X = analyzers.AnalyzePeptides(data=data.train_X, is_wide_format=False, ind_unstack='peptide')
# ana_train_X.df.set_index('peptide', append=True, inplace=True)
# ana_train_X.df.reset_index(inplace=True)
figures['pca_train'] = ana_train_X.plot_pca()

## Collaborative Filtering

## Data in wide format

- Autoencoder need data in wide format

In [ ]:
data.to_wide_format()
data.val_X.head()

## Denoising Autoencoder

### DataLoaders

In [ ]:
dae_default_pipeline = sklearn.pipeline.Pipeline(
    [
        ('normalize', StandardScaler()),
        ('impute', SimpleImputer(add_indicator=False))
    ])

dae_transforms = VaepPipeline(
    df_train=data.train_X, encode=dae_default_pipeline, decode=['normalize'])

dls = get_dls(data.train_X, data.val_X, transformer=dae_transforms)

### Model

In [ ]:
M = data.train_X.shape[-1]
latent_dim = 30

model = ae.Autoencoder(n_features=M, n_neurons=int(
    M/2), last_decoder_activation=None, dim_latent=latent_dim)

### Learner

In [ ]:
learn = Learner(dls=dls, model=model,
                loss_func=MSELossFlat(), cbs=ae.ModelAdapter())

In [ ]:
learn.show_training_loop()

In [ ]:
learn.summary()

In [ ]:
suggested_lr = learn.lr_find()
suggested_lr

### Training


In [ ]:
learn.fit_one_cycle(args.epochs_max, lr_max=suggested_lr.valley)

### Predictions
- test dataset

In [ ]:
# dls.test_dl
# needs to be part of setup procedure of a class
from vaep.io.datasets import DatasetWithTarget
def factory_test_dl(bs=64):
    
    def get_test_dl(df, transformer, dataset):
        ds = dataset(df, transformer)
        return DataLoader(ds, bs=bs)
    return get_test_dl

get_test_dl = factory_test_dl()
dl_test = get_test_dl(df=data.test_X, transformer=dae_transforms, dataset=DatasetWithTarget)

In [ ]:
pred, target = learn.get_preds(act=noop, concat_dim=0, reorder=False)
len(pred), len(target)

In [ ]:
pred, target = learn.get_preds(dl=dl_test, act=noop, concat_dim=0, reorder=False)
len(pred), len(target)

In [ ]:
"""
Returns
-------
    (pred, target)
"""
def get_preds_from_df(df, learn, transformer, dataset=DatasetWithTarget):
    dl = get_test_dl(df=df, transformer=transformer, dataset=dataset)
    res = learn.get_preds(dl=dl_test, concat_dim=0, reorder=False)
    res = L(res).map(lambda x: pd.DataFrame(x, index=df.index, columns=df.columns))
    res = L(res).map(lambda x: transformer.inverse_transform(x))
    return res

res = get_preds_from_df(df=data.test_X, learn=learn, transformer=dae_transforms)

# list(map(lambda x: x.shape, res))
L(res).map(lambda x: x.shape)

Some sanity checks 
- needs to go into test

In [ ]:
pred, target = res
assert len(data.test_X) == len(pred) == len(target)

In [ ]:
all(dl_test.dataset.mask_obs == data.test_X.isna())

In [ ]:
# should this be a function?
test_pred_observed_dae = pred[~dl_test.dataset.mask_obs].stack()
test_pred_real_na_dae = pred[dl_test.dataset.mask_obs].stack()
assert len(test_pred_real_na_dae) + len(test_pred_observed_dae) == reduce(mul, data.test_X.shape)

_diff_index = test_pred_real_na_dae.index.difference(test_pred_real_na.index)
if len(_diff_index):   
    print("Some predictions couldn't be generated using the approach using artifical replicates.\n"
         "These will be omitted for evaluation.")
    for _index in _diff_index:
        print(f"{_index[0]:<40}\t {_index[1]:<40}")
        
test_pred_observed['DAE'] = test_pred_observed_dae
test_pred_real_na['DAE'] = test_pred_real_na_dae

### Plots

## Comparison

In [ ]:
test_pred_observed.head()

In [ ]:
# df_pred = test_pred_observed
# y_true = df_pred.columns[0] # implicit assumption, default None
# y_true = df_pred.iloc[:,0]
# pred_columns = df_pred.columns[1:]

import sklearn.metrics as sklm

scoring = [('MSE', sklm.mean_squared_error),
           ('MAE', sklm.mean_absolute_error)]

def get_metrics_df(pred_df, true_col=None, scoring=scoring):
    if not true_col:
        # assume first column is truth if None is given
        y_true = pred_df.iloc[:,0]
        print(f'Selected as truth to compare to: {y_true.name}')
        y_pred = pred_df.iloc[:,1:]
    else:
        if issubclass(type(true_col), int):
            y_true = pred_df.iloc[:, true_col]
            pred_df = pred_df.drop(y_true.name, axis=1)
        elif issubclass(type(true_col), str):
            y_true = pred_df[true_col]
            pred_df = pred_df.drop(true_col, axis=1)   
        else:
            raise ValueError(f'true_col has to be of type str or int, not {type(true_col)}')
    

    metrics = {}
    for model_key in y_pred:
        model_pred = y_pred[model_key]
        model_pred_no_na = model_pred.dropna()
        if len(model_pred) > len(model_pred_no_na):
            logger.info(
                f"Drop indices for {col}: {[(idx[0], idx[1]) for idx in model_pred.index.difference(model_pred_no_na.index)]}")

        metrics[model_key] = dict(
            [(k, f(y_true=y_true.loc[model_pred_no_na.index], y_pred=model_pred_no_na))
             for k, f in scoring]
        )
    metrics = pd.DataFrame(metrics)
    return metrics
# metrics.to_csv(folder / f'exp_02_metrics.csv',
#                float_format='{:.3f}'.format)
metrics = get_metrics_df(pred_df = test_pred_observed)
metrics.sort_values(by=[k for k, f in scoring], axis=1)

In [ ]:
metrics = get_metrics_df(pred_df = test_pred_real_na)
metrics.sort_values(by=[k for k, f in scoring], axis=1)

In [ ]:
# analysis per sample?
# analysis per peptide?
# (test_pred_real_na['replicated'] - test_pred_real_na['DAE']).sort_values().plot(rot=45)

## Config

In [ ]:
args